In [1]:
using StaticArrays
# using Plots
using LinearAlgebra
# using Makie
using GLMakie 
using LsqFit
include("Elastic.jl")
using .Elastic
using FFMPEG

In [2]:
# 定义铜的晶格常数（单位：Å）
lattice_constant = 1.5

# 定义铜的FCC晶胞的基矢量
lattice_vectors = collect((Matrix([
    lattice_constant 0.0 0.0; #a1
    0.0 lattice_constant 0.0; #a2
    0.0 0.0 lattice_constant] #a3
))')

# 定义铜的FCC晶胞中的原子位置（单位：Å）
atom_positions = [
    Vector([0.0, 0.0, 0.0]),
    Vector([0.0, 0.5, 0.5]),
    Vector([0.5, 0.0, 0.5]),
    Vector([0.5, 0.5, 0.0]),
    Vector([1.0, 0.0, 0.0]),
    Vector([0.0, 1.0, 0.0]),
    Vector([0.0, 0.0, 1.0]),
    Vector([0.5, 1.0, 0.5]),
    Vector([1.0, 0.5, 0.5]),
    Vector([0.5, 0.5, 1.0]),
    Vector([1.0, 0.0, 1.0]),
    Vector([1.0, 1.0, 0.0]),
    Vector([0.0, 1.0, 1.0]),
    Vector([1.0, 1.0, 1.0])
] 

# 创建铜的原子列表
atoms = [Atom(pos) for pos in atom_positions]

cell=UnitCell(lattice_vectors,atoms)
cpcell=copycell(cell,3,3,3)
fig=visualize_unitcell_atoms(cpcell)

In [3]:
#lj势能
function lj(r::Float64)
    return 4*(1/r^12-1/r^6)
end
function Flj(r::Vector{Float64})
    rn=norm(r)
    return 24*(2/rn^14-1/rn^8)*r
end

ct=5.0
interaction = Interaction(lj, Flj, ct, 0.1)

x=1:0.001:ct
y=interaction.cutenergy.(x)
lines(x,y)

In [4]:
el=Vector{Float64}([])
lattice_constant = 3.61
cl=1.3:0.02:2.5
en=0
fn=0
fl=Vector{Float64}([])
for lattice_constant in cl
    # 定义铜的FCC晶胞的基矢量
    lattice_vectors = (Matrix([
        lattice_constant 0.0 0.0; #a1
        0.0 lattice_constant 0.0; #a2
        0.0 0.0 lattice_constant] #a3
    ))
    # 创建铜的原子列表
    atoms = [Atom(pos) for pos in atom_positions]

    cell=UnitCell(lattice_vectors,atoms)
    cpcell=copycell(cell,3,3,3)
    en=cell_energy0(cpcell,interaction,ifnormalize=true)
    push!(el,en)
    # fn=sum((cell_energy(cpcell,interaction)).^2)
    # push!(fl,fn)
    
end


In [5]:
# 创建一个新的 Figure
fig = Figure()

# 创建一个 Axis，并设置坐标范围
ax = Axis(fig[1, 1], title = "Line Plot", xlabel = "X-Axis", ylabel = "Y-Axis",
          )
vl=cl.^3*(cpcell.copy[1]*cpcell.copy[2]*cpcell.copy[3]);
# lines!(ax,cl,fl)
lines!(ax,cl,el)
# ylims!(ax, -0.001, 0.001)

fig

In [6]:
_,fig=BMfit(vl,el,cpcell)
fig

Fitted parameters:
V0 = 326.7246123157035
B0 = -5.046009356822658
B0' = 2.820280876351377
E0 = -141.49284306089007
latticeconstant = 2.295828074709017


[13676:7644:1005/200811.349:ERROR:t_skinfilemap.cpp(63)] SGRenderLog:CheckUpdate newMem.OpenMem Suc, szNewMem:Local\sgime_skin.Dpi144.v3.filemap.sogouime
[13676:7644:1005/200811.349:ERROR:t_skinfilemap.cpp(65)] SGRenderLog:CheckUpdate finish: old version:1
[13676:7644:1005/200811.349:ERROR:t_skinfilemap.cpp(67)] SGRenderLog:CheckUpdate finish: new version:3
[13676:7644:1005/200811.397:ERROR:t_skinfilemap.cpp(63)] SGRenderLog:CheckUpdate newMem.OpenMem Suc, szNewMem:Local\sgime_skin.Dpi144.v3.filemap.sogouime
[13676:7644:1005/200811.397:ERROR:t_skinfilemap.cpp(65)] SGRenderLog:CheckUpdate finish: old version:3
[13676:7644:1005/200811.398:ERROR:t_skinfilemap.cpp(67)] SGRenderLog:CheckUpdate finish: new version:3


In [7]:
thermostat = Thermostat(300.0, 1000.0, 0.1, 0.0)
barostat=Barostat(0.0,cell.Volume,1000.0,0.0)
z=cell2z(cpcell,thermostat,barostat);

In [ ]:
zl=[]
for i in 1:10000
    z=RK3_step(z,0.01,cpcell,interaction,thermostat,barostat)
    push!(zl,z)
end

In [19]:
newcell=z2cell(zl[2],lattice_vectors)
fig=visualize_unitcell_atoms0(newcell)

In [ ]:

# 假设func(i)是一个生成3D图的函数
function func(i)
    newcell=z2cell(zl[i],lattice_vectors)
    fig=visualize_unitcell_atoms0(newcell)
    return fig
end

function create_animation()
    video_file = "outputatom.mp4"
    n_frames = 1000
    fps = 1
    # 使用record函数生成动画
    record(fig, video_file, 1:n_frames; framerate = fps) do i
        func(i)
    end
end


# 调用函数生成视频
create_animation()